# Diffusion Example

In [ ]:
# dependencies
using Pkg
Pkg.activate(".")
Pkg.instantiate()
Pkg.develop(path="../..")
using Plots
using LFAToolkit
using LinearAlgebra

In [ ]:
# setup
p = 2
dimension = 1

mesh = []
if dimension == 1
   mesh = Mesh1D(1.0)
elseif dimension == 2
   mesh = Mesh2D(1.0, 1.0)
end

# operator
diffusion = GalleryOperator("diffusion", p + 1, p + 1, mesh)

In [ ]:
# compute full operator symbols
numbersteps = 100

# compute and plot smoothing factor
# -- 1D --
if dimension == 1
    # compute
    (θ_range, eigenvalues, _) = computesymbolsoverrange(diffusion, numbersteps)
    maxeigenvalues = maximum(real(eigenvalues); dims=2)
    mineigenvalues = minimum(real(eigenvalues); dims=2)

    # plot
    println("max eigenvalue: ", maximum(maxeigenvalues))
    xrange = θ_range/π
    plot(
        xrange,
        xlabel="θ/π",
        xtickfont=font(12, "Courier"),
        [maxeigenvalues, mineigenvalues],
        ytickfont=font(12, "Courier"),
        ylabel="spectral radius",
        linewidth=3,
        legend=:none,
        title="Diffusion Operator Symbol Maximal Eigenvalues"
    )
    ymin = minimum(mineigenvalues)
    ylims!(minimum([0, ymin * 1.1]), maximum(maxeigenvalues) * 1.1)
# -- 2D --
elseif dimension == 2
    # compute
    (_, eigenvalues, _) = computesymbolsoverrange(diffusion, numbersteps)
    maxeigenvalues = reshape(maximum(real(eigenvalues); dims=2), (numbersteps, numbersteps))
    mineigenvalues = reshape(minimum(real(eigenvalues); dims=2), (numbersteps, numbersteps))
    
    # plot
    θ_min = -π/2
    θ_max = 3π/2
    θ_range = LinRange(θ_min, θ_max, numbersteps)
    println("max eigenvalue: ", maximum(maxeigenvalues))
    xrange = θ_range/π
    plot1 = heatmap(
        xrange,
        xlabel="θ/π",
        xtickfont=font(12, "Courier"),
        xrange,
        ylabel="θ/π",
        maxeigenvalues,
        ytickfont=font(12, "Courier"),
        title="Diffusion Operator Symbol Maximum Eigenvalues",
        transpose=true,
        aspect_ratio=:equal
    )
    xlims!(θ_min/π, θ_max/π)
    ylims!(θ_min/π, θ_max/π)
    plot2 = heatmap(
        xrange,
        xlabel="θ/π",
        xtickfont=font(12, "Courier"),
        xrange,
        ylabel="θ/π",
        mineigenvalues,
        ytickfont=font(12, "Courier"),
        title="Diffusion Operator Symbol Minimum Eigevalues",
        transpose=true,
        aspect_ratio=:equal
    )
    xlims!(θ_min/π, θ_max/π)
    ylims!(θ_min/π, θ_max/π)
    plot!(plot1, plot2, layout = (2, 1), size = (700, 1400))
end